In [ ]:
from importlib.util import find_spec
in_colab = find_spec("google.colab") is not None

if in_colab:
    %cd /content
    !git clone https://github.com/EleutherAI/attribute
    %cd attribute
    !curl -LsSf https://astral.sh/uv/install.sh | sh
    !uv pip install -e .

In [ ]:
import serve
serve.main().launch(share=True)